Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

RuntimeError: module compiled against API version 0xc but this version of numpy is 0x9

RuntimeError: module compiled against API version 0xc but this version of numpy is 0x9

ValueError: numpy.dtype has the wrong size, try recompiling. Expected 88, got 96

First reload the data we generated in _notmist.ipynb_.

In [63]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [81]:
#print the reshaped data and sizes before reshaping
image_size = 28
num_labels = 10
def reformat_check(train_dataset_check,train_labels_check):
    train_dataset_check =train_dataset_check.reshape(-1,image_size*image_size)
    train_labels_check = train_labels_check
    return train_dataset_check,train_labels_check
#pickle_file_check = 'notMNIST.pickle'
#pickle_open_check = pickle.load(open(pickle_file_check,"rb"))
kk = np.linspace(1,10)
print(kk)
train_dataset_check = pickle_open_check['train_dataset']
train_labels_check = pickle_open_check['train_labels']
print('train_dataset_check_shape',train_dataset_check.shape )
print('train_labels_check_shape',train_labels_check.shape )
train_dataset_check,train_labels_check =  reformat_check(train_dataset_check,train_labels_check)
print('reformatted: data size',train_dataset_check.shape, 'label size' ,train_labels_check.shape)

print('check:',np.arange(10).shape,(train_labels_check[None,0:5]) ) # train_labels_check[:,None]

[  1.           1.18367347   1.36734694   1.55102041   1.73469388
   1.91836735   2.10204082   2.28571429   2.46938776   2.65306122
   2.83673469   3.02040816   3.20408163   3.3877551    3.57142857
   3.75510204   3.93877551   4.12244898   4.30612245   4.48979592
   4.67346939   4.85714286   5.04081633   5.2244898    5.40816327
   5.59183673   5.7755102    5.95918367   6.14285714   6.32653061
   6.51020408   6.69387755   6.87755102   7.06122449   7.24489796
   7.42857143   7.6122449    7.79591837   7.97959184   8.16326531
   8.34693878   8.53061224   8.71428571   8.89795918   9.08163265
   9.26530612   9.44897959   9.63265306   9.81632653  10.        ]
train_dataset_check_shape (200000, 28, 28)
train_labels_check_shape (200000,)
reformatted: data size (200000, 784) label size (200000,)
check: (10,) [[4 9 6 2 7]]


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [65]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [66]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [82]:
#Regularization on Logistic Regression model
#SGDescent method - Logistic regression model from assgnment 2
#Let's see the impact of regularization on the logistic regression model with SGDescent method
#Build the computation graph
from IPython.display import display
batch_size = 128
reg_param = 0.005
learn_init = 0.5
learn_fin = 0.001
num_steps = 3001
learn_rate = np.linspace(learn_init,learn_fin,num_steps)
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    #Adding the norm of weights and biases to the loss to penalize large weights - #regularization
  l2_loss_reg = reg_param*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss_reg
  
  # Optimizer.
  count = 0
  optimizer = tf.train.GradientDescentOptimizer(learn_rate[count]).minimize(loss)
  count = count + 1

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [83]:
#num_steps = 3001 
def accuracy(predictions, labels): 
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]) 

with tf.Session(graph=graph) as session: 
    tf.initialize_all_variables().run() 
    print("Initialized") 
    for step in range(num_steps):

        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step == 3000):
            print(predictions[1,:])
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 34.377686
Minibatch accuracy: 3.1%
Validation accuracy: 7.0%
Minibatch loss at step 500: 1.700962
Minibatch accuracy: 82.0%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 0.916723
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 0.911640
Minibatch accuracy: 78.1%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.725527
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.802202
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
[  2.55367928e-03   5.17735705e-02   2.02151225e-03   7.76054919e-01
   6.14695810e-03   6.13438751e-05   4.66922717e-03   1.46844715e-01
   9.34273005e-03   5.31395606e-04]
Minibatch loss at step 3000: 0.759323
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Test accuracy: 87.6%


In [91]:
#Regularization - Neural Net
#DeepLearning - 1 Layer deep Neural net with ReLU activation and softmax applied on the last layer
#Let's see the impact of regularization on the Neural Net
#Build the computation graph
from IPython.display import display
batch_size = 128
reg_param_hid = 0.005
reg_param_op = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #Hidden Layer variables
  hid_nodes = 1024
  weights_hid = tf.Variable(
              tf.truncated_normal([image_size * image_size, hid_nodes]))
  biases_hid = tf.Variable(tf.zeros([hid_nodes]))
  logits_hid = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hid) + biases_hid)
  display(tf_train_dataset)
  display(weights_hid)
  #keep_prob = tf.placeholder(tf.float32)
  #logits_hid_drop = tf.nn.dropout(logits_hid, keep_prob)
    
  weights_op = tf.Variable(
  tf.truncated_normal([hid_nodes, num_labels]))
  biases_op = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = (tf.matmul(logits_hid, weights_op) + biases_op)
  l2_reg_loss = (reg_param_hid*(tf.nn.l2_loss(weights_hid) + tf.nn.l2_loss(biases_hid)) 
                           +reg_param_op*( tf.nn.l2_loss(weights_op) + tf.nn.l2_loss(biases_op)))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_reg_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)
  test_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_test_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)

<tf.Tensor 'Placeholder:0' shape=(128, 784) dtype=float32>

In [92]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #display(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2004.459717
Minibatch accuracy: 3.9%
Validation accuracy: 31.7%
Minibatch loss at step 500: 126.281265
Minibatch accuracy: 84.4%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 10.939548
Minibatch accuracy: 82.0%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 1.607610
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.775031
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.781914
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.688162
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [97]:
#Regularization - Overfitting for Neural Net model
#DeepLearning - 1 Layer deep Neural net with ReLU activation and softmax applied on the last layer
#Let's see the impact of regularization on the Neural Net
#Build the computation graph
from IPython.display import display
batch_size = 64  #Overfitting extreme case - small batch size and smaller epochs
reg_param_hid = 0.005
reg_param_op = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #Hidden Layer variables
  hid_nodes = 1024
  weights_hid = tf.Variable(
              tf.truncated_normal([image_size * image_size, hid_nodes]))
  biases_hid = tf.Variable(tf.zeros([hid_nodes]))
  logits_hid = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hid) + biases_hid)
  display(tf_train_dataset)
  display(weights_hid)
  #keep_prob = tf.placeholder(tf.float32)
  #logits_hid_drop = tf.nn.dropout(logits_hid, keep_prob)
    
  weights_op = tf.Variable(
  tf.truncated_normal([hid_nodes, num_labels]))
  biases_op = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = (tf.matmul(logits_hid, weights_op) + biases_op)
  l2_reg_loss = (reg_param_hid*(tf.nn.l2_loss(weights_hid) + tf.nn.l2_loss(biases_hid)) 
                           +reg_param_op*( tf.nn.l2_loss(weights_op) + tf.nn.l2_loss(biases_op)))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_reg_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)
  test_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_test_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)

<tf.Tensor 'Placeholder:0' shape=(64, 784) dtype=float32>

In [98]:
num_steps = 501 #Extreme case of overfitting - smaller batch size and number of steps

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #display(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1908.673462
Minibatch accuracy: 4.7%
Validation accuracy: 25.5%
Minibatch loss at step 500: 127.559990
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Test accuracy: 88.4%


In [ ]:
#Observation: When regularization is applied and the training data (num of batches) is small then there is
#no benefit of applying the regularization. Rather if the data is more then the accuracy can improve a lot.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [106]:
#Regularization + Dropout
#DeepLearning - 1 Layer deep Neural net with ReLU activation and softmax applied on the last layer (Rugularized)
#Let's see the impact of dropout on the Neural Net
#Build the computation graph
from IPython.display import display
batch_size = 128  
#Regularization
reg_param_hid = 0.005
reg_param_op = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Dropput_Keep probability
  keep_prob = tf.placeholder(tf.float32)
  # Variables.
  #Hidden Layer variables
  hid_nodes = 1024
  weights_hid = tf.Variable(
              tf.truncated_normal([image_size * image_size, hid_nodes]))
  biases_hid = tf.Variable(tf.zeros([hid_nodes]))
  logits_hid = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hid) + biases_hid)
  #Adding dropout
  logits_hid_drop = tf.nn.dropout(logits_hid,keep_prob)
  
  weights_op = tf.Variable(
  tf.truncated_normal([hid_nodes, num_labels]))
  biases_op = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = (tf.matmul(logits_hid_drop, weights_op) + biases_op)

  l2_reg_loss = (reg_param_hid*(tf.nn.l2_loss(weights_hid) + tf.nn.l2_loss(biases_hid)) 
                           +reg_param_op*( tf.nn.l2_loss(weights_op) + tf.nn.l2_loss(biases_op)))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_reg_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)
  test_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_test_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)

In [107]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #display(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2138.922852
Minibatch accuracy: 7.8%
Validation accuracy: 31.2%
Minibatch loss at step 500: 127.773697
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 11.073450
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 1.709401
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.850884
Minibatch accuracy: 79.7%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.818401
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.762805
Minibatch accuracy: 80.5%
Validation accuracy: 83.7%
Test accuracy: 90.6%


In [108]:
#Regularization + Dropout (Extreme Overfit case)
#DeepLearning - 1 Layer deep Neural net with ReLU activation and softmax applied on the last layer (Rugularized)
#Let's see the impact of dropout on the Neural Net
#Build the computation graph
from IPython.display import display
batch_size = 80  #Overfitting extreme case - small batch size and smaller epochs
#Regularization
reg_param_hid = 0.005
reg_param_op = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Dropput_Keep probability
  keep_prob = tf.placeholder(tf.float32)
  # Variables.
  #Hidden Layer variables
  hid_nodes = 1024
  weights_hid = tf.Variable(
              tf.truncated_normal([image_size * image_size, hid_nodes]))
  biases_hid = tf.Variable(tf.zeros([hid_nodes]))
  logits_hid = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hid) + biases_hid)
  #Adding dropout
  logits_hid_drop = tf.nn.dropout(logits_hid,keep_prob)
  
  weights_op = tf.Variable(
  tf.truncated_normal([hid_nodes, num_labels]))
  biases_op = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = (tf.matmul(logits_hid_drop, weights_op) + biases_op)

  l2_reg_loss = (reg_param_hid*(tf.nn.l2_loss(weights_hid) + tf.nn.l2_loss(biases_hid)) 
                           +reg_param_op*( tf.nn.l2_loss(weights_op) + tf.nn.l2_loss(biases_op)))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_reg_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)
  test_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_test_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)

In [109]:
num_steps = 301

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #display(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2128.245117
Minibatch accuracy: 11.2%
Validation accuracy: 29.1%
Test accuracy: 86.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [210]:
#Regularization + Dropout + Improve the accuracy
#DeepLearning - 1 Layer deep Neural net with ReLU activation and softmax applied on the last layer (Rugularized)
#Let's see the impact of dropout on the Neural Net
#Build the computation graph
from IPython.display import display
batch_size = 128  
#Regularization
reg_param_hid = 0.005
reg_param_op = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  learn_rate = tf.placeholder(tf.float32)
  #Dropput_Keep probability
  keep_prob = tf.placeholder(tf.float32)
  # Variables.
  #Hidden Layer variables
  hid_nodes = 1500
  weights_hid = tf.Variable(
              tf.truncated_normal([image_size * image_size, hid_nodes]))
  biases_hid = tf.Variable(tf.zeros([hid_nodes]))
  logits_hid = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hid) + biases_hid)
  #Adding dropout
  logits_hid_drop = tf.nn.dropout(logits_hid,keep_prob)
  
  weights_op = tf.Variable(
  tf.truncated_normal([hid_nodes, num_labels]))
  biases_op = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = (tf.matmul(logits_hid_drop, weights_op) + biases_op)

  l2_reg_loss = (reg_param_hid*(tf.nn.l2_loss(weights_hid) + tf.nn.l2_loss(biases_hid)) 
                           +reg_param_op*( tf.nn.l2_loss(weights_op) + tf.nn.l2_loss(biases_op)))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_reg_loss
  
  # Optimizer.

  optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.6, global_step,10,0.9)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)
  test_prediction = tf.nn.softmax(tf.matmul(
                        tf.nn.relu(tf.matmul(tf_test_dataset, weights_hid) + biases_hid),
                                    weights_op) + biases_op)

In [211]:
num_steps = 3001
learnrate_init = 0.4
learnrate_final = 0.001
learn_rate_eval = np.linspace(learnrate_init,learnrate_final,num_steps) #Decreasing learning rate
#display(learn_rate_eval)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #display(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                 keep_prob : 0.8, learn_rate : learn_rate_eval[step] }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2782.816406
Minibatch accuracy: 12.5%
Validation accuracy: 27.6%
Minibatch loss at step 500: 364.341248
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 80.865723
Minibatch accuracy: 84.4%
Validation accuracy: 84.8%
Minibatch loss at step 1500: 25.524073
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 11.348359
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 7.145287
Minibatch accuracy: 82.8%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 6.079482
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Test accuracy: 93.2%
